# 데이터 가져오기

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
import InceptionV4

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((124, 124)),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


# dataloder, optimizer, loss

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = InceptionV4.InceptionV4(124, 10)
model.to(device)

cuda


InceptionV4(
  (conv): Sequential(
    (0): InceptionV4StemBlock(
      (conv1): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU()
      )
      (maxpool2_1): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv2_1): Sequential(
        (0): Conv2d(64, 96, kernel_size=(3, 3), stride=(2, 2))
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv3_l): Sequential(
        (0): C

In [9]:
from torch.utils.data import DataLoader

batch_size = 128
epochs = 30

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for batch, labels in train_dataloader:
    print(batch.shape, labels.shape)
    break

torch.Size([128, 3, 124, 124]) torch.Size([128])


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1)

# 훈련 및 평가

In [13]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0.0
    with torch.no_grad():
        for batches, labels in test_loader:
            batches = batches.to(device)
            labels = labels.to(device)
            outputs = model(batches)
            test_loss += criterion(outputs, labels).item()
            prediction = outputs.max(1, keepdim=True)[1]
            correct += prediction.eq(labels.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy



def train_one_epoch(epoch_number):
    print(f"epoch{epoch_number} start")
    print("#####################################################")
    model.train()
    running_loss = 0.0
    last_loss = 0.0
    correct = 0

    for i, data in enumerate(train_dataloader):
        batches, labels = data
        batches = batches.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(batches)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        pred = torch.argmax(outputs, dim=1)
        correct = correct + torch.sum(pred == labels).item()
        if i % 100 == 99:
            acc = 100. * correct / ((i+1) * batch_size)

            last_loss = running_loss / ((i+1) * batch_size)
            print('  train {} loss: {}'.format(i + 1, last_loss))
            print('  train {} accuracy: {} %\n'.format(i+1, acc))

    return last_loss, acc

In [15]:
train_log = {'loss': [], 'acc': []}
test_log = {'loss': [], 'acc': []}


for i in range(10):
    train_loss, train_acc = train_one_epoch(i+1)
    test_loss, test_acc = evaluate(model, test_dataloader)
    schedular.step(test_loss)
    print(f"\n[EPOCH: {i+1}]\tTest Loss: {test_loss:.4f}\tTest Accuracy: {test_acc} % ")
    print("#####################################################\n\n")

    train_log['loss'].append(train_loss)
    train_log['acc'].append(train_acc)
    test_log['loss'].append(test_loss)
    test_log['acc'].append(test_acc)

epoch1 start
#####################################################
  train 100 loss: 0.01619829689152539
  train 100 accuracy: 21.2421875 %

  train 200 loss: 0.01610235315747559
  train 200 accuracy: 21.30078125 %

  train 300 loss: 0.015987162307525674
  train 300 accuracy: 21.7578125 %


[EPOCH: 1]	Test Loss: 0.0220	Test Accuracy: 23.39 % 
#####################################################


epoch2 start
#####################################################
  train 100 loss: 0.015203998647630215
  train 100 accuracy: 24.3203125 %

  train 200 loss: 0.015163672715425491
  train 200 accuracy: 24.8359375 %

  train 300 loss: 0.015231153337905805
  train 300 accuracy: 24.747395833333332 %


[EPOCH: 2]	Test Loss: 0.0155	Test Accuracy: 30.11 % 
#####################################################


epoch3 start
#####################################################
  train 100 loss: 0.014659892655909062
  train 100 accuracy: 28.65625 %

  train 200 loss: 0.01470938631799072
  train 200